## Task 2 - Naive CUDA Kernel

In [1]:
%%writefile matrix_gpu_naive.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

__global__ void matrixMultiplyGPU(float *A, float *B, float *C, int N) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    if (row < N && col < N) {
        float sum = 0.0f;
        for (int k = 0; k < N; k++) {
            sum += A[row * N + k] * B[k * N + col];
        }
        C[row * N + col] = sum;
    }
}

int main(int argc, char **argv) {
    int N = 512;
    if (argc > 1) {
        N = atoi(argv[1]);
    }

    size_t matrix_size = N * N * sizeof(float);
    
    float *host_A = (float *)malloc(matrix_size);
    float *host_B = (float *)malloc(matrix_size);
    float *host_C = (float *)malloc(matrix_size);
    
    for (int i = 0; i < N * N; i++) {
        host_A[i] = rand() % 100 / 100.0f;
        host_B[i] = rand() % 100 / 100.0f;
    }    
    float *device_A;
    float *device_B;
    float *device_C;
    
    cudaMalloc((void**)&device_A, matrix_size);
    cudaMalloc((void**)&device_B, matrix_size);
    cudaMalloc((void**)&device_C, matrix_size);
    
    cudaMemcpy(device_A, host_A, matrix_size, cudaMemcpyHostToDevice);
    cudaMemcpy(device_B, host_B, matrix_size, cudaMemcpyHostToDevice);
    
    dim3 threadsPerBlock(16, 16);
    
    // Calculate how many blocks we need
    int num_blocks = (N + 15) / 16;
    dim3 numBlocks(num_blocks, num_blocks);
    
    printf("Using %d x %d blocks with 16 x 16 threads per block\n", num_blocks, num_blocks);
    
    cudaEvent_t start_event, stop_event;
    cudaEventCreate(&start_event);
    cudaEventCreate(&stop_event);
    
    //start time
    cudaEventRecord(start_event);    
    matrixMultiplyGPU<<<numBlocks, threadsPerBlock>>>(device_A, device_B, device_C, N);
    cudaEventRecord(stop_event); //stop time
    cudaEventSynchronize(stop_event);
    
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start_event, stop_event);
    
    printf("Naive CUDA execution time (N=%d): %f ms\n", N, milliseconds);
    printf("In %f seconds\n", milliseconds / 1000.0f);
    
    cudaMemcpy(host_C, device_C, matrix_size, cudaMemcpyDeviceToHost);
    
    volatile float prevent_optimization = host_C[0];
    
    cudaFree(device_A);
    cudaFree(device_B);
    cudaFree(device_C);
    
    free(host_A);
    free(host_B);
    free(host_C);

    return 0;
}

Overwriting matrix_gpu_naive.cu


## Task 3 - Tiled CUDA code

In [2]:
!nvidia-smi

Sat Jan 31 03:50:51 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [4]:
!nvcc matrix_gpu_naive.cu -o matrix_gpu_naive

In [5]:
!./matrix_gpu_naive 512
!./matrix_gpu_naive 1024
!./matrix_gpu_naive 2048
!./matrix_gpu_naive 4096
!./matrix_gpu_naive 8192

Using 32 x 32 blocks with 16 x 16 threads per block
Naive CUDA execution time (N=512): 7.286528 ms
In 0.007287 seconds
Using 64 x 64 blocks with 16 x 16 threads per block
Naive CUDA execution time (N=1024): 7.479136 ms
In 0.007479 seconds
Using 128 x 128 blocks with 16 x 16 threads per block
Naive CUDA execution time (N=2048): 7.086944 ms
In 0.007087 seconds
Using 256 x 256 blocks with 16 x 16 threads per block
Naive CUDA execution time (N=4096): 7.450912 ms
In 0.007451 seconds
Using 512 x 512 blocks with 16 x 16 threads per block
Naive CUDA execution time (N=8192): 7.147232 ms
In 0.007147 seconds


## Task 4 - Optimizing CUDA Code

In [6]:
%%writefile matrix_gpu_tiled.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#define TILE_WIDTH 16

__global__ void matrixMultiplyTiled(float *A, float *B, float *C, int N) {
    __shared__ float ds_A[TILE_WIDTH][TILE_WIDTH];
    __shared__ float ds_B[TILE_WIDTH][TILE_WIDTH];

    int bx = blockIdx.x;
    int by = blockIdx.y;
    int tx = threadIdx.x;
    int ty = threadIdx.y;

    int Row = by * TILE_WIDTH + ty;
    int Col = bx * TILE_WIDTH + tx;

    float Pvalue = 0.0f;

    for (int m = 0; m < (N + TILE_WIDTH - 1) / TILE_WIDTH; ++m) {

        if (Row < N && (m * TILE_WIDTH + tx) < N)
            ds_A[ty][tx] = A[Row * N + m * TILE_WIDTH + tx];
        else
            ds_A[ty][tx] = 0.0f;

        if (Col < N && (m * TILE_WIDTH + ty) < N)
            ds_B[ty][tx] = B[(m * TILE_WIDTH + ty) * N + Col];
        else
            ds_B[ty][tx] = 0.0f;

        __syncthreads();

        for (int k = 0; k < TILE_WIDTH; ++k)
            Pvalue += ds_A[ty][k] * ds_B[k][tx];

        __syncthreads();
    }

    if (Row < N && Col < N)
        C[Row * N + Col] = Pvalue;
}

int main(int argc, char **argv) {
    int N = 512;
    if (argc > 1) {
        N = atoi(argv[1]);
    }

    size_t matrix_size = N * N * sizeof(float);
    
    float *host_A = (float *)malloc(matrix_size);
    float *host_B = (float *)malloc(matrix_size);
    float *host_C = (float *)malloc(matrix_size);
    
    for (int i = 0; i < N * N; i++) {
        host_A[i] = rand() % 100 / 100.0f;
        host_B[i] = rand() % 100 / 100.0f;
    }    
    float *device_A;
    float *device_B;
    float *device_C;
    
    cudaMalloc((void**)&device_A, matrix_size);
    cudaMalloc((void**)&device_B, matrix_size);
    cudaMalloc((void**)&device_C, matrix_size);
    
    cudaMemcpy(device_A, host_A, matrix_size, cudaMemcpyHostToDevice);
    cudaMemcpy(device_B, host_B, matrix_size, cudaMemcpyHostToDevice);
    
    dim3 threadsPerBlock(16, 16);
    
    // Calculate how many blocks we need
    int num_blocks = (N + 15) / 16;
    dim3 numBlocks(num_blocks, num_blocks);
    
    printf("Using %d x %d blocks with 16 x 16 threads per block\n", num_blocks, num_blocks);
    
    cudaEvent_t start_event, stop_event;
    cudaEventCreate(&start_event);
    cudaEventCreate(&stop_event);
    
    //start time
    cudaEventRecord(start_event);    
    matrixMultiplyTiled<<<numBlocks, threadsPerBlock>>>(device_A, device_B, device_C, N);
    cudaEventRecord(stop_event); //stop time
    cudaEventSynchronize(stop_event);
    
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start_event, stop_event);
    
    printf("Tiled CUDA execution time (N=%d): %f ms\n", N, milliseconds);
    printf("In %f seconds\n", milliseconds / 1000.0f);
    
    cudaMemcpy(host_C, device_C, matrix_size, cudaMemcpyDeviceToHost);
    
    volatile float prevent_optimization = host_C[0];
    
    cudaFree(device_A);
    cudaFree(device_B);
    cudaFree(device_C);
    
    free(host_A);
    free(host_B);
    free(host_C);

    return 0;
}

Overwriting matrix_gpu_tiled.cu


In [7]:
!nvcc matrix_gpu_tiled.cu -o matrix_gpu_tiled

In [8]:
!./matrix_gpu_tiled 512
!./matrix_gpu_tiled 1024
!./matrix_gpu_tiled 2048
!./matrix_gpu_tiled 4096
!./matrix_gpu_tiled 8192

Using 32 x 32 blocks with 16 x 16 threads per block
Tiled CUDA execution time (N=512): 7.083840 ms
In 0.007084 seconds
Using 64 x 64 blocks with 16 x 16 threads per block
Tiled CUDA execution time (N=1024): 7.318496 ms
In 0.007318 seconds
Using 128 x 128 blocks with 16 x 16 threads per block
Tiled CUDA execution time (N=2048): 7.219872 ms
In 0.007220 seconds
Using 256 x 256 blocks with 16 x 16 threads per block
Tiled CUDA execution time (N=4096): 7.092448 ms
In 0.007092 seconds
Using 512 x 512 blocks with 16 x 16 threads per block
Tiled CUDA execution time (N=8192): 7.253760 ms
In 0.007254 seconds


## Task 5 - Performance Comparison

In [9]:
# data
python_512 = 7.81 * 1000
python_1024 = 65.46 * 1000
python_2048 = 839.49 * 1000

cpu_512  = 0.49 * 1000
cpu_1024 = 3.71 * 1000
cpu_2048 = 37.63 * 1000

naive_512  = 0.0125 * 1000
naive_1024 = 0.0109 * 1000
naive_2048 = 0.0107 * 1000

tiled_512  = 0.0088 * 1000
tiled_1024 = 0.0106 * 1000
tiled_2048 = 0.0109 * 1000


In [10]:
speedup_naive_512  = cpu_512  / naive_512
speedup_naive_1024 = cpu_1024 / naive_1024
speedup_naive_2048 = cpu_2048 / naive_2048

speedup_tiled_512  = cpu_512  / tiled_512
speedup_tiled_1024 = cpu_1024 / tiled_1024
speedup_tiled_2048 = cpu_2048 / tiled_2048

In [11]:
import pandas as pd

data = {
    "Implementation": [
        "CPU Time (ms)",
        "Naive CUDA Time (ms)",
        "Naive CUDA Speedup",
        "Tiled CUDA Time (ms)",
        "Tiled CUDA Speedup"
    ],
    "N = 512": [
        cpu_512,
        naive_512,
        speedup_naive_512,
        tiled_512,
        speedup_tiled_512
    ],
    "N = 1024": [
        cpu_1024,
        naive_1024,
        speedup_naive_1024,
        tiled_1024,
        speedup_tiled_1024
    ],
    "N = 2048": [
        cpu_2048,
        naive_2048,
        speedup_naive_2048,
        tiled_2048,
        speedup_tiled_2048
    ]
}

df = pd.DataFrame(data)
pd.options.display.float_format = "{:.2f}".format
df

,Implementation,N = 512,N = 1024,N = 2048
0,CPU Time (ms),490.00,3710.00,37630.00
1,Naive CUDA Time (ms),12.50,10.90,10.70
2,Naive CUDA Speedup,39.20,340.37,3516.82
3,Tiled CUDA Time (ms),8.80,10.60,10.90
4,Tiled CUDA Speedup,55.68,350.00,3452.29


## Step 6 - Using cuBLAS Library

In [12]:
%%writefile matrix_gpu_cublas.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <cublas_v2.h>

int main(int argc, char **argv) {
    int N;
    if (argc > 1) {
        N = atoi(argv[1]);
    } else {
        N = 1024; // default size
    }
    
    size_t size = N * N * sizeof(float);

    float *h_A = (float *)malloc(size);
    float *h_B = (float *)malloc(size);
    float *h_C = (float *)malloc(size);

    for (int i = 0; i < N * N; i++) {
        h_A[i] = rand() % 100 / 100.0f;
    }
    
    for (int i = 0; i < N * N; i++) {
        h_B[i] = rand() % 100 / 100.0f;
    }

    float *d_A;
    float *d_B;
    float *d_C;
    
    cudaMalloc((void**)&d_A, size);
    cudaMalloc((void**)&d_B, size);
    cudaMalloc((void**)&d_C, size);

    cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);

    cublasHandle_t handle;
    cublasCreate(&handle);

    float alpha = 1.0f;
    float beta  = 0.0f;

    cudaEvent_t start;
    cudaEvent_t stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);
    cublasSgemm(handle,
                CUBLAS_OP_N, CUBLAS_OP_N,
                N, N, N,
                &alpha,
                d_B, N,
                d_A, N,
                &beta,
                d_C, N);

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float ms;
    cudaEventElapsedTime(&ms, start, stop);
    printf("cuBLAS SGEMM time (N=%d): %f ms\n", N, ms);

    cudaMemcpy(h_C, d_C, size, cudaMemcpyDeviceToHost);   
    volatile float sink = h_C[0];

    cublasDestroy(handle);
    
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    free(h_A);
    free(h_B);
    free(h_C);

    return 0;
}

Overwriting matrix_gpu_cublas.cu


In [13]:
!nvcc matrix_gpu_tiled.cu -o matrix_gpu_tiled

In [14]:
!./matrix_gpu_tiled 512
!./matrix_gpu_tiled 1024
!./matrix_gpu_tiled 2048

Using 32 x 32 blocks with 16 x 16 threads per block
Tiled CUDA execution time (N=512): 7.873824 ms
In 0.007874 seconds
Using 64 x 64 blocks with 16 x 16 threads per block
Tiled CUDA execution time (N=1024): 7.152192 ms
In 0.007152 seconds
Using 128 x 128 blocks with 16 x 16 threads per block
Tiled CUDA execution time (N=2048): 7.178624 ms
In 0.007179 seconds


## Step 7 - Creating a Shared Library and Using it in Python

In [15]:
%%writefile matrix_lib.cu
#include <cuda_runtime.h>
#include <stdio.h>
#define TILE_WIDTH 16

__global__ void matrixMultiplyTiled(float *A, float *B, float *C, int N) {
    __shared__ float ds_A[TILE_WIDTH][TILE_WIDTH];
    __shared__ float ds_B[TILE_WIDTH][TILE_WIDTH];

    int tx = threadIdx.x;
    int ty = threadIdx.y;
    
    int Row = blockIdx.y * TILE_WIDTH + ty;
    int Col = blockIdx.x * TILE_WIDTH + tx;

    float Pvalue = 0.0f;

    int numTiles = (N + TILE_WIDTH - 1) / TILE_WIDTH;
    for (int m = 0; m < numTiles; ++m) {
        int aCol = m * TILE_WIDTH + tx;
        if (Row < N && aCol < N) {
            ds_A[ty][tx] = A[Row * N + aCol];
        } else {
            ds_A[ty][tx] = 0.0f;
        }

        int bRow = m * TILE_WIDTH + ty;
        if (Col < N && bRow < N) {
            ds_B[ty][tx] = B[bRow * N + Col];
        } else {
            ds_B[ty][tx] = 0.0f;
        }

        __syncthreads();

        for (int k = 0; k < TILE_WIDTH; ++k) {
            Pvalue += ds_A[ty][k] * ds_B[k][tx];
        }

        __syncthreads();
    }

    if (Row < N && Col < N) {
        C[Row * N + Col] = Pvalue;
    }
}

extern "C" void gpu_matrix_multiply(float *h_A, float *h_B, float *h_C, int N) {
    size_t size = N * N * sizeof(float);

    float *d_A;
    float *d_B;
    float *d_C;
    
    cudaMalloc((void**)&d_A, size);
    cudaMalloc((void**)&d_B, size);
    cudaMalloc((void**)&d_C, size);

    cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);

    dim3 block(TILE_WIDTH, TILE_WIDTH);
    
    int numBlocksX = (N + TILE_WIDTH - 1) / TILE_WIDTH;
    int numBlocksY = (N + TILE_WIDTH - 1) / TILE_WIDTH;
    dim3 grid(numBlocksX, numBlocksY);

    matrixMultiplyTiled<<<grid, block>>>(d_A, d_B, d_C, N);
    cudaDeviceSynchronize();
    cudaMemcpy(h_C, d_C, size, cudaMemcpyDeviceToHost);

    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
}

Overwriting matrix_lib.cu


In [16]:
!nvcc -Xcompiler -fPIC -shared matrix_lib.cu -o libmatrix.so

In [17]:
import ctypes
import numpy as np
import time

lib = ctypes.cdll.LoadLibrary("./libmatrix.so")
lib.gpu_matrix_multiply.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.float32, ndim=1, flags="C_CONTIGUOUS"),
    np.ctypeslib.ndpointer(dtype=np.float32, ndim=1, flags="C_CONTIGUOUS"),
    np.ctypeslib.ndpointer(dtype=np.float32, ndim=1, flags="C_CONTIGUOUS"),
    ctypes.c_int
]

N = 512

for i in range(5):
    print(f"\nTesting with matrix size N = {N}")
    
    A = np.random.rand(N, N).astype(np.float32)
    B = np.random.rand(N, N).astype(np.float32)
    C = np.zeros((N, N), dtype=np.float32)
    
    start = time.time()
    lib.gpu_matrix_multiply(A.ravel(), B.ravel(), C.ravel(), N)
    end = time.time()
    elapsed_time = end - start
    
    print(f"CUDA library time (N={N}): {elapsed_time * 1000:.4f} milliseconds")
    
    N = N * 2


Testing with matrix size N = 512
CUDA library time (N=512): 128.7131 milliseconds

Testing with matrix size N = 1024
CUDA library time (N=1024): 4.2226 milliseconds

Testing with matrix size N = 2048
CUDA library time (N=2048): 21.9338 milliseconds

Testing with matrix size N = 4096
CUDA library time (N=4096): 87.1000 milliseconds

Testing with matrix size N = 8192
CUDA library time (N=8192): 286.8943 milliseconds


## Step 8 - Adding Custom Functions to the Shared Library

In [22]:
%%writefile conv_gpu.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

__global__ void convolution2D_GPU(
    unsigned char *image,
    int *kernel,
    int *output,
    int width,
    int height
) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x >= width || y >= height) return;

    int sum = 0;
    int k = 1;  // 3x3 kernel radius

    for (int ky = -k; ky <= k; ky++) {
        for (int kx = -k; kx <= k; kx++) {
            int ix = x + kx;
            int iy = y + ky;
            if (ix >= 0 && ix < width && iy >= 0 && iy < height) {
                int pixel = image[iy * width + ix];
                int weight = kernel[(ky + k) * 3 + (kx + k)];
                sum += pixel * weight;
            }
        }
    }
    output[y * width + x] = sum;
}

int main() {
    int width = 512;
    int height = 512;
    size_t img_size = width * height * sizeof(unsigned char);
    size_t out_size = width * height * sizeof(int);

    unsigned char *h_img = (unsigned char*)malloc(img_size);
    int *h_out = (int*)malloc(out_size);

    int h_kernel[9] = {
        -1, -1, -1,
        -1,  8, -1,
        -1, -1, -1
    };

    for (int i = 0; i < width * height; i++)
        h_img[i] = rand() % 256;

    unsigned char *d_img;
    int *d_kernel, *d_out;

    cudaMalloc(&d_img, img_size);
    cudaMalloc(&d_kernel, 9 * sizeof(int));
    cudaMalloc(&d_out, out_size);

    cudaMemcpy(d_img, h_img, img_size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_kernel, h_kernel, 9 * sizeof(int), cudaMemcpyHostToDevice);

    dim3 block(16, 16);
    dim3 grid((width + 15) / 16, (height + 15) / 16);

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cudaEventRecord(start);
    convolution2D_GPU<<<grid, block>>>(d_img, d_kernel, d_out, width, height);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    float ms;
    cudaEventElapsedTime(&ms, start, stop);
    printf("CUDA convolution time: %f ms\n", ms);

    cudaMemcpy(h_out, d_out, out_size, cudaMemcpyDeviceToHost);

    cudaFree(d_img);
    cudaFree(d_kernel);
    cudaFree(d_out);
    free(h_img);
    free(h_out);

    return 0;
}


Overwriting conv_gpu.cu


In [23]:
!nvcc conv_gpu.cu -o conv_gpu

In [25]:
!./conv_gpu

CUDA convolution time: 7.416800 ms
